# CONVERT FILES WITH DUCK_DB

In [1]:
import pandas as pd
import duckdb
import os
import time
import numpy as np
from utils import time, data_processor, constants

In [2]:
OUTPUT_PATH = r"..\data_processed\\guo_subs_europee\\subs_eu\\" 

eu27_countries = constants.eu

## Ownership history

In [7]:
TEMP_TABLE_FIRMOGRAPHICS = "..\\data_processed\\firmographics_processed\\*.parquet" 
TEMP_TABLE_KEY_FINANCIALS = "..\\data_processed\\firmographics_historicals_processed"


def get_ownership_data(year, country, path=None):
    print(f"{country} - {year}...")
    COUNTRY_KEY_FINANCIALS = TEMP_TABLE_KEY_FINANCIALS + f"\\firmographics_historicals_{country}.parquet"
            
            
    query = f"""
        SELECT
            main.subsidiary_bvd_id,
            main.guo_25,
            firmographics_sub.nuts2 AS subsidiary_nuts2,
            firmographics_sub.nace_rev_2_core_code_4_digits_ AS subsidiary_nace4,
            firmographics_guo.nuts2 AS guo_nuts2,
            firmographics_guo.nace_rev_2_core_code_4_digits_ AS guo_nace4,
            firmographics_guo.type_of_entity AS guo_type_of_entity,
            firmographics_guo.status AS guo_status,
            {year} AS year,
            key_financials.number_of_employees AS guo_number_of_employees,
            key_financials.closing_date AS guo_closing_date
        FROM 
            '{path}' AS main
        LEFT JOIN 
            '{TEMP_TABLE_FIRMOGRAPHICS}' AS firmographics_sub
        ON 
            main.subsidiary_bvd_id = firmographics_sub.bvd_id_number
        LEFT JOIN
            '{TEMP_TABLE_FIRMOGRAPHICS}' AS firmographics_guo
        ON
            main.guo_25 = firmographics_guo.bvd_id_number
        LEFT JOIN 
            (SELECT * 
             FROM '{COUNTRY_KEY_FINANCIALS}'
             WHERE 
                year = {year}
            ) AS key_financials
        ON
            main.guo_25 = key_financials.bvd_id_number   -- CHANGE HERE FOR SUBS
        AND
            key_financials.year = {year}

        WHERE 
            main."type_of_relation" = 'GUO 25'
        AND 
            main."guo_25" LIKE '{country}%'
    """
    

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()
    return df



In [8]:
@time.time_it
def fetch_and_convert_to_excel(year, country, path, output_path):
    df = get_ownership_data(year, country, path)
    os.makedirs(output_path, exist_ok=True)
    slices = data_processor.split_dataframe(df, 5)
    
    for i, s in enumerate(slices):
        s.to_csv(f"{output_path}\\{country}_{year}_chunk_{i+1}.csv", index=False)
        print(f"Data for {country} - {year} - slice_{i+1} has been converted.")
        

### Test

In [ ]:
# path = "..\\data_raw\\ownership_history\\links_2007\\*.parquet"
country = "DE"
year = "2009"
path = "..\\data_raw\\ownership_history\\links_2007\\*.parquet"

df = get_ownership_data(
    path=path,
    year=year,
    country=country,
)
# fetch_and_convert_to_excel(country=country, year=year, path=path, output_path=output_path)

## Query data

In [ ]:
import time
import json

errors = {}

total_start = time.time()
start_time = time.time()

for country in eu27_countries:
    for year in range(2022, 2023):
        year = str(year)
        try:
            path = f"..\\data_raw\\ownership_history\\links_{year}\\*.parquet"
            fetch_and_convert_to_excel(year, country, path, OUTPUT_PATH)  
            end_time = time.time()
            duration = end_time - start_time
            print(f"Data for {country} - {year} has been converted to Stata in {duration:.2f} seconds.")
        except Exception as e:
            errors[f"{country} - {year}"] = str(e)
            print(f"Error {country} - {year}")
            continue


end_time = time.time()
duration = end_time - start_time
total_end = time.time()
total_duration = total_end - total_start
print(f"Total execution time: {total_duration:.2f} seconds.")

In [ ]:
import json 

with open("errors/errors_guos.json", "r") as f:
    errors = json.load(f)

## Zip the files

In [16]:
ZIP_OUTPOUT = "..\\zipped_files\\guos_eu"

data_processor.compress_csv_files(folder_to_zip=OUTPUT_PATH, zip_output=ZIP_OUTPOUT)